In [6]:
import sklearn

In [7]:
import numpy as np

In [8]:
import pandas as pd

In [9]:
import chardet

In [11]:
with open("/home/swarnadeep/Documents/Courses/2nd_Sem/AML/SMSSpamCollection","rb") as f:
    result = chardet.detect(f.read())
print(result['encoding'])
df_raw = pd.read_csv("/home/swarnadeep/Documents/Courses/2nd_Sem/AML/SMSSpamCollection",sep='\t',encoding=result['encoding'])

utf-8


In [12]:
df_raw.head()#change the first line of the source file to v1 and v2

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [13]:
df=df_raw[['v1','v2']].rename(columns={'v1':'label','v2':'sms'})

In [14]:
df.head()

,label,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [15]:
df.describe()

,label,sms
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [16]:
df.drop_duplicates(subset="sms",inplace=True)

In [17]:
df.describe()

,label,sms
count,5169,5169
unique,2,5169
top,ham,There is a first time for everything :)
freq,4516,1


In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X=df['sms']

In [20]:
y=df['label']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [22]:
print("Shape of X is {}".format(X.shape))
print("Shape of X_train is {} and shape of y_train is {}".format(X_train.shape, y_train.shape))
print("Shape of X_test is {} and shape of y_test is {}".format(X_test.shape, y_test.shape))

Shape of X is (5169,)
Shape of X_train is (3876,) and shape of y_train is (3876,)
Shape of X_test is (1293,) and shape of y_test is (1293,)


In [23]:
train_corpus = list(X_train)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
vectorizer = TfidfVectorizer(max_features=5000)
vectorizer.fit(train_corpus)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [26]:
print("Number of features = {}".format(len(vectorizer.vocabulary_)))
print("Number of omitted words = {}".format(len(vectorizer.stop_words_)))

Number of features = 5000
Number of omitted words = 2277


In [27]:
X_train_text_features = vectorizer.transform(list(X_train))
print("Shape of X_train_text_features is {}".format(X_train_text_features.shape))

Shape of X_train_text_features is (3876, 5000)


In [28]:
df['len'] = df['sms'].map(lambda x: len(x))

In [29]:
X = df[['sms', 'len']]
y = df['label']

In [30]:
X_train, X_test, y_train, y_test =  train_test_split(X, y, random_state=42)

In [31]:
train_corpus = list(X_train['sms'])

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [33]:
vectorizer = TfidfVectorizer(max_features=5000)
vectorizer.fit(train_corpus)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [34]:
from scipy import sparse

In [35]:
#need to be evaluated.. and to understand theory
def get_features(X):
    X_text_features = vectorizer.transform(list(X['sms']))
    X_len_features = sparse.csr_matrix(X['len']).T
    X_features = sparse.hstack([X_text_features, X_len_features])
    return X_features

In [36]:
X_train_features = get_features(X_train)

In [37]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=42)

model.fit(X_train_features, y_train)

/home/swarnadeep/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [38]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [39]:
y_train_predicted = model.predict(X_train_features)

In [40]:
print("The fraction of correctly classified samples is {}".format(accuracy_score(y_train, y_train_predicted)))
print("The number of correctly classified samples is {}".format(accuracy_score(y_train, 
                                                                               y_train_predicted, normalize=False)))

The fraction of correctly classified samples is 0.9770381836945304
The number of correctly classified samples is 3787


In [45]:
pd.DataFrame(confusion_matrix(y_train_predicted, y_train), 
             index={'true spam', 'true ham'}, 
             columns={'pred spam','pred ham', })

,pred ham,pred spam
true spam,3382,79
true ham,10,405


In [46]:
X_test_features = get_features(X_test)
y_test_predicted = model.predict(X_test_features)

In [47]:
print("The fraction of correctly classified samples is {}".format(accuracy_score(y_test, y_test_predicted)))
print("The number of correctly classified samples is {}".format(accuracy_score(y_test, y_test_predicted, normalize=False)))

pd.DataFrame(confusion_matrix(y_test_predicted, y_test), 
             index={'true ham', 'true spam'}, 
             columns={'pred ham', 'pred spam'})

The fraction of correctly classified samples is 0.9729311678267595
The number of correctly classified samples is 1258


,pred ham,pred spam
true spam,1121,32
true ham,3,137


In [54]:
from sklearn.model_selection import cross_val_score

In [55]:
lr_model = LogisticRegression()
cv_scores = cross_val_score(lr_model, X=X_train_features, y=y_train, cv=5, n_jobs=4)
print(cv_scores)

[0.96520619 0.96134021 0.96387097 0.97032258 0.95478036]


In [48]:
from sklearn.naive_bayes import MultinomialNB

In [50]:
model = MultinomialNB()

In [51]:
model.fit(X_train_features, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [52]:
y_train_predicted = model.predict(X_train_features)

In [53]:
print("The fraction of correctly classified samples is {}".format(accuracy_score(y_train, y_train_predicted)))
print("The number of correctly classified samples is {}".format(accuracy_score(y_train, 
                                                                               y_train_predicted, normalize=False)))

The fraction of correctly classified samples is 0.901186790505676
The number of correctly classified samples is 3493


In [56]:
bayes_model = MultinomialNB()
cv_scores = cross_val_score(bayes_model, X=X_train_features, y=y_train, cv=5, n_jobs=4)
print(cv_scores)

[0.875      0.87886598 0.87741935 0.87741935 0.87596899]


In [57]:
from sklearn.tree import DecisionTreeClassifier

In [58]:
classifier = DecisionTreeClassifier()  
classifier.fit(X_train_features, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [59]:
decision_tree = DecisionTreeClassifier()
cv_scores = cross_val_score(decision_tree, X=X_train_features, y=y_train, cv=5, n_jobs=4)
print(cv_scores)

[0.97164948 0.95360825 0.95612903 0.95612903 0.9625323 ]


In [60]:
from sklearn.ensemble import RandomForestClassifier

In [62]:
forest = RandomForestClassifier(random_state=42)
forest.fit(X_train_features, y_train)

/home/swarnadeep/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [63]:
cv_scores = cross_val_score(forest, X=X_train_features, y=y_train, cv=5, n_jobs=4)
print(cv_scores)

[0.97036082 0.96907216 0.96774194 0.96774194 0.96640827]
